In [1]:
from scrapy import Selector
import requests
import sqlite3
import pandas as pd
from lets_plot import *
LetsPlot.setup_html()

In [2]:
def premier_league(base_url):

    try:
        html = requests.get(base_url)
        html.raise_for_status()
        sel = Selector(text=html.content)
    except requests.RequestException as e:
        print(f"Error fetching {base_url}: {e}")

    match_df = []
    
    containers = sel.css("div.club-cards-wrapper.indexSection li")
    for container in containers:
        # Extract team data
        Data = {
            'team_name': container.css("div.club-card__name-container h2::text").get(),
            'home_stadium': container.css("div.club-card__name-container span::text").get()
        }
        
        # Extract links and create individual team URL
        cleaned_links = [link.replace("/clubs", "") for link in container.css("a::attr(href)").getall()]
        cleaned_link = cleaned_links[0] if cleaned_links else None
        
        if not cleaned_link:
            print(f"No link found for team: {Data['team_name']}")
            continue
        
        ind_team_url = f'{base_url + cleaned_link.replace("overview", "stats")}'
        
        # Fetch individual team stats page
        try:
            team_response = requests.get(ind_team_url)
            team_response.raise_for_status()
            team_sel = Selector(text=team_response.content)
        except requests.RequestException as e:
            print(f"Error fetching {ind_team_url}: {e}")
            continue
        
        # Extract stats
        match_play = team_sel.css("div.all-stats__top-list span::text").getall()
        cl = [item.strip() for item in match_play if item.strip()]
        
        raw = team_sel.css("div.all-stats__list-container div.all-stats__regular-stat-container span::text").extract()
        cleaned_data = [item.strip() for item in raw if item.strip()]
        
        # Combine match and additional stats
        plays = {
            'matches_play': cl[0],
            'wins': cl[1],
            'losses': cl[2]
        }
        plays.update({cleaned_data[i]: cleaned_data[i+1] for i in range(0, len(cleaned_data), 2)})
        
        # Combine all data and append to the list
        Data.update(plays)
        match_df.append(Data)

        for row in match_df:
            update_row = {key.replace('%','in_per').replace(' ','_').replace('/','_or_'): value for key, value in row.items()}
            row.clear()
            row.update(update_row)
        
    # Convert to DataFrame and return
    return match_df
   


def db_storage(match_df,table_name,db_name):

    conn = sqlite3.connect(f'DB/{db_name}')
    cursor = conn.cursor()

    if not match_df:
        print("No Data to insert")
        return
    
    first_row = match_df[0]
    columns = []

    for key, value in first_row.items():
        if isinstance(value,int):
            data_type = "INTEGER"
        elif isinstance(value,float):
            data_type = "REAL"
        else:
            data_type = "TEXT"
        columns.append(f"{key} {data_type}")

    create_query_table = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(columns)});"
    cursor.execute(create_query_table)
    conn.commit()

    for row in match_df:
        place_holders = ', '.join('?' for i in row)
        insert_query = f"INSERT INTO {table_name} ({', '.join(row.keys())}) VALUES ({place_holders})"
        cursor.execute(insert_query,tuple(row.values()))
    
        conn.commit()
    
    conn.close()
    print("Stored!")
    return

In [3]:
if __name__ == "__main__":
    base_url =  "https://www.premierleague.com/clubs"
    database = premier_league(base_url=base_url)

    if database:
        db_storage(database,table_name="Premier_league",db_name="Scrape.db")

Stored!


In [3]:
db_name = "Scrape.db"
conn = sqlite3.connect(f'DB/{db_name}')
cursor = conn.cursor()

In [4]:
pd.read_sql_query('''SELECT * FROM Premier_league''',conn) # To check Data is loaded properly

,team_name,home_stadium,matches_play,wins,losses,Goals,Goals_per_match,Shots,Shots_on_target,Shooting_accuracy_in_per,...,Interceptions,Clearances,Headed_Clearance,Aerial_Battles_or_Duels_Won,Errors_leading_to_goal,Own_goals,Yellow_cards,Red_cards,Fouls,Offsides
0,Arsenal,Emirates Stadium,"1,508",839,334,"2,741",1.82,"14,008","5,076",36%,...,"12,561","21,021","8,653","63,041",178,64,"2,263",122,"3,545","2,074"
1,Aston Villa,Villa Park,"1,228",450,451,"1,572",1.28,"8,134","2,757",34%,...,"8,362","18,699","6,702","44,600",121,60,"1,938",75,"2,308","1,343"
2,Bournemouth,Vitality Stadium,457,178,187,652,1.43,"4,325","1,496",35%,...,"3,665","8,052","4,013","22,451",65,20,760,23,"1,771",537
3,Brentford,Gtech Community Stadium,312,132,110,501,1.61,"2,367",913,39%,...,"1,693","3,738","1,800","11,735",30,11,423,15,941,349
4,Brighton & Hove Albion,American Express Stadium,468,170,163,617,1.32,"4,740","1,596",34%,...,"3,470","6,468","3,400","22,816",52,31,780,35,"2,160",697
5,Chelsea,Stamford Bridge,"1,574",867,352,"2,805",1.78,"16,191","5,758",36%,...,"11,756","21,546","8,847","66,682",135,53,"2,505",108,"3,911","2,290"
6,Crystal Palace,Selhurst Park,786,257,325,941,1.2,"6,027","2,055",34%,...,"6,416","12,381","6,373","38,828",81,32,"1,203",39,"3,256",930
7,Everton,Goodison Park,"1,352",501,493,"1,777",1.31,"10,337","3,533",34%,...,"9,745","23,000","8,908","54,626",138,65,"2,111",115,"3,060","1,653"
8,Fulham,Craven Cottage,834,283,344,"1,043",1.25,"6,785","2,244",33%,...,"6,833","16,233","5,479","33,708",82,49,"1,309",60,"1,513","1,087"
9,Ipswich Town,Portman Road,581,207,223,749,1.29,955,341,36%,...,495,"1,239",607,"3,798",15,17,817,32,422,156


In [5]:
conn.close()

In [ ]:
premier_league_df = pd.DataFrame(database)
premier_league_df.head(3)

In [222]:
numeric_col_int = ['matches_play','wins','losses','Goals','Shots','Shots_on_target','Penalties_scored','Big_Chances_Created',
                    'Hit_woodwork','Passes','Crosses','Clean_sheets','Goals_Conceded','Saves','Tackles','Blocked_shots','Interceptions',
                    'Clearances','Headed_Clearance','Aerial_Battles_or_Duels_Won', 'Errors_leading_to_goal', 'Own_goals',
                    'Yellow_cards', 'Red_cards', 'Fouls', 'Offsides']

numeric_col_flo = ['Goals_per_match','Passes_per_match','Goals_conceded_per_match']

In [234]:
premier_league_df[numeric_col_int] = premier_league_df[numeric_col_int].apply(lambda s: s.str.replace(',','').astype(int))
premier_league_df[numeric_col_flo] = premier_league_df[numeric_col_flo].astype(float)

In [270]:
premier_league_df.to_csv("Data\Premier_league.csv",index=False)

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\admin\AppData\Local\Temp\ipykernel_11692\27699778.py:1: SyntaxWarning: invalid escape sequence '\P'
  premier_league_df.to_csv("Data\Premier_league.csv",index=False)


In [20]:
premier_league_df = pd.read_csv('Data/Premier_league.csv')

In [7]:
premier_league_df.head()

,team_name,home_stadium,matches_play,wins,losses,Goals,Goals_per_match,Shots,Shots_on_target,Shooting_accuracy_in_per,...,Interceptions,Clearances,Headed_Clearance,Aerial_Battles_or_Duels_Won,Errors_leading_to_goal,Own_goals,Yellow_cards,Red_cards,Fouls,Offsides
0,Arsenal,Emirates Stadium,1508,839,334,2741,1.82,14008,5076,36%,...,12562,21021,8653,63039,178,64,2263,122,3545,2074
1,Aston Villa,Villa Park,1228,450,451,1572,1.28,8134,2757,34%,...,8362,18699,6702,44600,121,60,1938,75,2308,1343
2,Bournemouth,Vitality Stadium,456,178,187,650,1.43,4318,1493,35%,...,3655,8025,4005,22392,65,20,757,23,1765,536
3,Brentford,Gtech Community Stadium,311,132,110,499,1.60,2349,907,39%,...,1683,3717,1789,11688,30,11,423,15,941,345
4,Brighton & Hove Albion,American Express Stadium,468,170,163,617,1.32,4740,1596,34%,...,3470,6468,3400,22816,52,31,780,35,2160,697


In [101]:
tooltip = (layer_tooltips()
            .title('@team_name')
            .line('Total matches| @matches_play')
            .line('Goals| @Goals_per_match')
            .format('Goals_per_match','{} per match'))


team_goal_performance = (
ggplot(
    premier_league_df, aes(x ='team_name', color = 'Goals_per_match')
)+geom_line(aes(y ='matches_play'), tooltips=tooltip)+
scale_cmapmpl(cmap='plasma',aesthetic='color')+
labs(
    title="Team's Goal Performance",
    subtitle="Team's Number of Matches and their Goals per Match",
    x = "Teams",
    y = "Matches Play",
    color = "Goals per Match"
)+theme_void()+flavor_high_contrast_light()+
theme(text = element_text(family= 'mono',color='black',size = 15),
      plot_title= element_text(size = 20, face = 'bold'),
      plot_subtitle=element_text(size=18,face='italic'),
      axis_text_x = element_text(size = 14),
      axis_text_y= element_text(size = 12),
      axis_title_x = element_text(size = 18),
      axis_title_y= element_text(size = 18),
      legend_background=element_rect(size = 2)
      )
)+ggsize(800,500)

team_goal_performance
#ggsave(team_goal_performance,"Charts/team_goal_performance.svg")

In [104]:
tooltip = (layer_tooltips()
            .anchor('top_right')
            .title('@team_name')
            .line('Red cards| @Red_cards')
            .line('Fouls| @Fouls'))

team_negative_performance = (
    ggplot(premier_league_df, aes(x='team_name', y='Red_cards', fill='Fouls')) +
    geom_bar(stat='identity', tooltips=tooltip) +
    labs(
    title="Team's Negative Performance",
    subtitle="Team's Number of Red cards and their Fouls",
    x = "Teams",
    y = "Red cards",
    fill = "Fouls"
)+flavor_high_contrast_light()+theme_minimal()+
theme(text = element_text(family= 'mono', size = 15),
      plot_title= element_text(size = 20, face = 'bold'),
      plot_subtitle=element_text(size=18, face='italic'),
      axis_text_x = element_text(size = 14),
      axis_text_y= element_text(size = 12),
      axis_title_x = element_text(size = 18),
      axis_title_y= element_text(size = 18),
      legend_background=element_rect(size = .5)
      )
)+ggsize(800,500)
team_negative_performance
#ggsave(team_negative_performance,'Charts/team_negative_performance.svg')

In [106]:
premier_league_df_melted = premier_league_df.melt(id_vars=['team_name'], value_vars=['wins', 'losses'], var_name='result', value_name='count')

premier_league_df_melted['percentage'] = (
    premier_league_df_melted['count'] /
    premier_league_df_melted.groupby('team_name')['count'].transform('sum') * 100
)


premier_league_df_melted['result'] = premier_league_df_melted['result'].replace({'wins': 'Win', 'losses': 'Loss'})

tooltip = (layer_tooltips()
            .anchor('top_left')
            .title("@team_name")
            .line('@result - @percentage')
            .format('@percentage','{.2f}%')
            .format('@result','')
            )

team_overall_performance = (
    ggplot(premier_league_df_melted, aes(x='team_name', y='count', fill='result')) +
    geom_bar(stat='identity', position='fill', tooltips=tooltip) +
    scale_fill_manual(values=['#1f77b4', '#ff7f0e']) +
    labs(
    title="Team's Overall Performance",
    subtitle="Team's Number of Wins and Losses",
    x = "Teams",
    y = "Wins / Losses Ratio",
    fill = "Result"
)+flavor_high_contrast_light()+theme_minimal()+
theme(text = element_text(family= 'mono', size = 15),
      plot_title= element_text(size = 20, face = 'bold'),
      plot_subtitle=element_text(size=18, face='italic'),
      axis_text_x = element_text(size = 14),
      axis_text_y= element_text(size = 12),
      axis_title_x = element_text(size = 18),
      axis_title_y= element_text(size = 18),
      legend_background=element_rect(size = .5)
      )
)+ggsize(800,500)
team_overall_performance
#ggsave(team_overall_performance,'Charts/team_overall_performance.svg')